In [18]:
import bs4
import requests
import re
import tqdm

In [19]:
def get_string(c):
    return ''.join(i.string if i.string is not None else "" for i in c.children)

def convert(section):
    children = list(section.children)
    if len(children) < 2:
        return

    # print(section.attrs)
    # title = children[1].string
    # if title is None or len(title.split()) > 1:
    #     return

    title = section.attrs['id'].split('/')[-1]
    data = False
    if title.startswith('body.'):
        title = title[5:]
    if title == 'request_body':
        title = 'ComputeRoutesRequest' # todo
    if 'Response' in title:
        data = True

    # print(title)

    code_type = ""
    enum_vals = {}
    fields = {}

    for i in children[2:]:
        if i.name != 'section':
            continue

        if 'description' in i.attrs['id']:
            p = list(i.children)[1]
            description = get_string(p)
        elif 'ENUM_VALUES' in i.attrs['id']:
            code_type = "enum"
            for row in i.find("tbody").find_all("tr"):
                cs = list(row.children)
                td1, td2 = cs[1], cs[3]
                enum_vals[get_string(td1)] = get_string(td2)
        elif 'FIELDS' in i.attrs['id']:
            code_type = "struct"
            for row in i.find('tbody').find_all('tr'):
                cs = list(row.children)
                try:
                    td1, td2 = cs[1], cs[3]
                    cs2 = list(td2.children)
                    cs3 = list(cs2[1].children)
                    fields[get_string(td1)] = (get_string(cs3[0]), get_string(cs2[3]))
                except:
                    # title += "\n" + get_string(cs[1])
                    pass # todo comment?

    return (title, code_type, enum_vals, fields, data)

def get_things(sections):
    things = {}

    for child in sections:
        if child.name == 'section' and hasattr(child, 'attrs') and 'id' in child.attrs:
            if vals := convert(child):
                t, ct, ev, f, d = vals
                things[t] = (ct, ev, f, d)
    return things

In [20]:
def convert_2(section):
    h3 = section.find("h3")
    if h3 is None:
        return

    title = h3.attrs['id']

    data = 'Response' in title

    code_type = ""
    enum_vals = {}
    fields = {}

    table = section.find("tbody")
    if table is None:
        return # todo enums
    code_type = "struct"
    for row in table.find_all("tr"):
        name, optional, type, comment = list(row.find_all("td"))
        name, type, comment = get_string(name).strip(), get_string(type).strip(), get_string(comment).strip()
        if type.startswith("Array"):
            type = type[6:-1]
            name += '[]'
        fields[name] = (type, comment.replace("\n", " "))

    return (title, code_type, enum_vals, fields, data)

def get_things_2(sections):
    things = {}

    for child in sections:
        if vals := convert_2(child):
            t, ct, ev, f, d = vals
            things[t] = (ct, ev, f, d)
    return things

In [21]:
def convert_type(name, type):
    if type == 'object' or re.match(r'object\ \(.*\ .*\)', type):
        t = '[String: Any]'
    elif type.startswith('object') or type.startswith('enum'):
        t = type.split('(')[1][:-1]
    elif type.startswith('string'):
        t = 'String'
    elif type == 'integer':
        t = 'Int'
    elif type == 'number':
        t = 'Double'
    elif type == 'boolean':
        t = 'Bool'
    else:
        print("unknown type", type)
        t = type
    n = name
    if name.endswith("[]"):
        n = n[:-2]
        t = '[' + t + ']'
    return n, t
        
def get_file(things):
    ss = ""
    for k, v in things.items():
        ct, ev, f, d = v
        s = ""
        if ct == "enum":
            s += f"enum {k}: String, Codable {{\n"
            for name, comment in ev.items():
                s += f"\tcase {name} // {comment}\n"
            s += "}"
        elif ct == "struct":
            s += f"struct {k}: Codable, Identifiable {{\n"
            if 'id' not in f:
                s += '\tvar id = UUID()\n"
            manual = False
            for name, (type, comment) in f.items():
                n, c = convert_type(name, type)
                if '[String: Any]' in c:
                    manual = True
                s += f"\tvar {n}: {c}? = nil // {comment}\n"
            if manual:
                s += "\n\tenum CodingKeys: String, CodingKey {\n"
                for name in f.keys():
                    s += f"\t\tcase {name.strip("[]")}\n"
                s += "\t}\n\n"
                s += "\tinit(from decoder: Decoder) throws {\n\t\tlet container = try decoder.container(keyedBy: CodingKeys.self)\n\n"
                for name, (type, comment) in f.items():
                    n, c = convert_type(name, type)
                    s += f"\t\t{n} = try container.decodeIfPresent({c}.self, forKey: .{n})\n"
                s += "\t}\n\n\tfunc encode(to encoder: Encoder) throws {\n\t\tvar container = encoder.container(keyedBy: CodingKeys.self)\n\n"
                for name in f.keys():
                    n = name.strip("[]")
                    s += f"\t\ttry container.encodeIfPresent({n}, forKey: .{n})\n"
                s += "\t}\n"
            s += "}\n"
            if d:
                s += f"""
extension {k} {{
    static func from(jsonData data: Data) -> {k}? {{
        return try? JSONDecoder().decode({k}.self, from: data)
    }}
}}


"""
        if s != "":
            ss += s + "\n\n"
    return ss

In [22]:
def to_swift(url):
    soup = bs4.BeautifulSoup(requests.get(url).text)
    sections = soup.find_all("section", recursive=True)
    things = get_things(sections)
    return get_file(things)

def to_swift_2(url):
    soup = bs4.BeautifulSoup(requests.get(url).text)
    sections = soup.find_all("div", attrs={"data-code-snippet": True}, recursive=True)
    things = get_things_2(sections)
    return get_file(things)

In [23]:
urls = [
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TopLevel/computeRoutes"
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/FallbackInfo",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/LatLng",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/LocalizedText",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Location",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Money",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RouteModifiers",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RouteTravelAdvisory",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RouteTravelMode",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RoutingPreference",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/SpeedReadingInterval",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Status",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TollInfo",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TrafficModel",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TransitPreferences",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Waypoint"
]

sss = "import Foundation\n"
for u in tqdm.tqdm(urls):
    sss += to_swift(u) + "\n"

open("/Users/simonchervenak/Documents/GitHub/moev/moev/moev/API/Models/ComputeRoutes.swift", "w").write(sss)

100%|███████████████████████████████████████████| 15/15 [00:07<00:00,  2.04it/s]


46163

In [24]:
s = "import Foundation\n"
s += to_swift_2("https://developers.google.com/maps/documentation/places/web-service/autocomplete")
open("/Users/simonchervenak/Documents/GitHub/moev/moev/moev/API/Models/PlaceAutocomplete.swift", "w").write(s)

unknown type PlaceAutocompletePrediction
unknown type PlacesAutocompleteStatus
unknown type PlaceAutocompleteMatchedSubstring
unknown type PlaceAutocompleteStructuredFormat
unknown type PlaceAutocompleteTerm
unknown type PlaceAutocompleteMatchedSubstring
unknown type PlaceAutocompleteMatchedSubstring


2007

In [25]:
s = "import Foundation\n"
s += to_swift_2("https://developers.google.com/maps/documentation/places/web-service/details")
open("/Users/simonchervenak/Documents/GitHub/moev/moev/moev/API/Models/PlaceDetails.swift", "w").write(s)

unknown type Place
unknown type PlacesDetailsStatus
unknown type AddressComponent
unknown type PlaceOpeningHours
unknown type PlaceEditorialSummary
unknown type Geometry
unknown type PlaceOpeningHours
unknown type PlacePhoto
unknown type PlusCode
unknown type PlaceReview
unknown type PlaceOpeningHours
unknown type LatLngLiteral
unknown type Bounds
unknown type LatLngLiteral
unknown type LatLngLiteral
unknown type PlaceOpeningHoursPeriod
unknown type PlaceSpecialDay
unknown type PlaceOpeningHoursPeriodDetail
unknown type PlaceOpeningHoursPeriodDetail


9416